In [1]:
from pymongo import MongoClient
import pandas as pd

# Dowloading Tranning data from mongodb

In [2]:
client = MongoClient('localhost', 27017)
db=client.oncosteam
employee = db.train

In [3]:
train = []
for i in employee.find({}):
    train.append(i)
 
    

In [4]:
df = pd.DataFrame(train)

In [5]:
df

,_id,target,tumor size,node status of the tumor,Age of the patient,tumor grade
0,5eb02c6c9c55fdd7d1c0526d,0,2,1,60,3
1,5eb02c6c9c55fdd7d1c0526e,0,1,1,55,2
2,5eb02c6c9c55fdd7d1c0526f,0,1,1,33,3
3,5eb02c6c9c55fdd7d1c05270,0,1,1,67,2
4,5eb02c6c9c55fdd7d1c05271,0,1,1,47,2
...,...,...,...,...,...,...
229,5eb02c6c9c55fdd7d1c05352,1,2,2,55,2
230,5eb02c6c9c55fdd7d1c05353,1,3,2,45,2
231,5eb02c6c9c55fdd7d1c05354,1,3,2,37,3
232,5eb02c6c9c55fdd7d1c05355,1,3,2,58,2


# Preprocessing data 

In [6]:
from sklearn.model_selection import train_test_split

In [7]:
X_res = df.iloc[:,2:].values
y_res = df.iloc[:,1].values

In [8]:
y_res

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [9]:
from imblearn.over_sampling import RandomOverSampler

Using TensorFlow backend.


In [10]:
os = RandomOverSampler()
X_res,y_res = os.fit_sample(X_res,y_res)

In [11]:
X_res.shape

(382, 4)

In [12]:
y_res.shape

(382,)

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X_res,y_res, test_size=0.2, random_state=42)


# Traning Model 

In [14]:
from sklearn.preprocessing import Normalizer

In [15]:
scaler = Normalizer().fit(X_train)
standarized_x = scaler.transform(X_train)
standarized_x_test = scaler.transform(X_test)

In [16]:

from sklearn.metrics import accuracy_score,confusion_matrix,classification_report

In [17]:
import xgboost as xgb
xgb_model = xgb.XGBClassifier(objective='binary:logistic')
xgb_model.fit(standarized_x,y_train)

XGBClassifier(base_score=0.5, booster=None, colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints=None,
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=0, num_parallel_tree=1,
              objective='binary:logistic', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=1, subsample=1, tree_method=None,
              validate_parameters=False, verbosity=None)

In [18]:
y_pred = xgb_model.predict(standarized_x_test)
print(accuracy_score(y_test,y_pred))
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))

0.8311688311688312
[[35 12]
 [ 1 29]]
              precision    recall  f1-score   support

           0       0.97      0.74      0.84        47
           1       0.71      0.97      0.82        30

    accuracy                           0.83        77
   macro avg       0.84      0.86      0.83        77
weighted avg       0.87      0.83      0.83        77



# Saving tranined model to mongodb 

In [19]:
def save_model_to_db(model, client, db, dbconnection, model_name):
    import pickle
    import time
    import pymongo
    #pickling the model
    pickled_model = pickle.dumps(model)
    
    #saving model to mongoDB
    # creating connection
    myclient = pymongo.MongoClient(client)
    
    #creating database in mongodb
    mydb = myclient[db]
    
    #creating collection
    mycon = mydb[dbconnection]
    info = mycon.insert_one({model_name: pickled_model, 'name': model_name, 'created_time':time.time()})
    print(info.inserted_id, ' saved with this id successfully!')
    
    details = {
        'inserted_id':info.inserted_id,
        'model_name':model_name,
        'created_time':time.time()
    }
    
    return details

In [21]:
details = save_model_to_db(model = xgb_model, client ='mongodb://localhost:27017/', db = 'oncosteam', 
                 dbconnection = 'model', model_name = 'myxgb')

5eb037412c660dffc47ac84d  saved with this id successfully!


In [22]:
print(details)

{'inserted_id': ObjectId('5eb037412c660dffc47ac84d'), 'model_name': 'myxgb', 'created_time': 1588606785.1241617}
